In [10]:
# flash app with route to fetch weather data for canada from weatherapi.com
from flask import Flask, jsonify, render_template
import requests

app = Flask(__name__)

@app.route('/weatherd')
def weatherd():
    url = 'http://api.weatherapi.com/v1/current.json?key=<35307998dae54e14acd05652231502>&q=Canada'
    response = requests.get(url)
    data = response.json()['current']
    return jsonify(data)


In [13]:
# sqlite database and table for storing weather data
import sqlite3

#conn = sqlite3.connect('weatherd.db')
#c = conn.cursor()

#c.execute('''CREATE TABLE weatherd
             (id INTEGER PRIMARY KEY AUTOINCREMENT,
              city TEXT,
              state TEXT,
              country TEXT,
              precip_n FLOAT,
              pressure_in FLOAT,
              temp_c FLOAT,
              wind FLOAT)''')




In [20]:
# sqlite database and table for storing weather data
import sqlite3

conn = sqlite3.connect('weatherd.db')
c = conn.cursor()

c.execute('''CREATE TABLE weatherd
             (id INTEGER PRIMARY KEY AUTOINCREMENT,
              name TEXT,
              region TEXT,
              country TEXT,
              precip_n FLOAT,
              pressure_in FLOAT,
              temp_c FLOAT,
              wind FLOAT)''')

conn.commit()


In [22]:
# route to store data in sqlite database
@app.route('/weatherd')
def weather2():
    url = 'http://api.weatherapi.com/v1/current.json?key=<Y35307998dae54e14acd05652231502>&q=Canada'
    response = requests.get(url)
    data = response.json()['current']
    
    # Insert data into SQLite database
    c.execute('''INSERT INTO weatherd
                 (name, region, country, precip_n, pressure_in, temp_c, wind)
                 VALUES (?, ?, ?, ?, ?, ?, ?)''',
              (data['name'], data['region'], data['country'], 
               data['precip_mm'], data['pressure_in'], data['temp_c'], data['wind_kph']))
    conn.commit()
    
    return jsonify(data)


In [24]:
#plotly chart html
@app.route('/chart')
def chart2():
    c.execute('SELECT name, temp_c, wind FROM weatherd')
    data = c.fetchall()
    
    x_values = [row[0] for row in data]
    y_values1 = [row[1] for row in data]
    y_values2 = [row[2] for row in data]
    
    fig = {
        'data': [
            {'x': x_values, 'y': y_values1, 'type': 'bar', 'name': 'Temperature (C)'},
            {'x': x_values, 'y': y_values2, 'type': 'bar', 'name': 'Wind (kph)'}
        ],
        'layout': {
            'title': 'Weather Data for Canadian Cities',
            'xaxis': {'title': 'City'},
            'yaxis': {'title': 'Value'}
        }
    }
    
    return render_template('chart.html', plot=fig)


In [26]:
# dashboard

@app.route('/dashboard')
def dashboards():
    # Get weather data from SQLite database
    conn = sqlite3.connect('weatherd.db')
    df = pd.read_sql_query('SELECT * FROM weather', conn)
    conn.close()

    # Create Plotly charts
    chart1 = create_chart(df, 'temp_c', 'Temperature (C)')
    chart2 = create_chart(df, 'wind_kph', 'Wind (kph)')
    chart3 = create_chart(df, 'pressure_in', 'Pressure (in)')

    # Render dashboard page with charts
    return render_template('dashboard.html', 
                           chart1=chart1,
                           chart2=chart2,
                           chart3=chart3)
                           
def create_chart(df, column, title):
    data = [
        {'x': df['city'], 'y': df[column], 'type': 'bar'}
    ]
    
    layout = {
        'title': title,
        'xaxis': {'title': 'City'},
        'yaxis': {'title': title}
    }
    
    return {'data': data, 'layout': layout}
